In [121]:
import pandas as pd
import os
import sys

current_dir = os.getcwd()
src_path = os.path.join(current_dir, '..')
sys.path.append(src_path)

from src.financials.financial_report import FinancialReport
from src.financials.pme_metrics import PmeTable, PmeMetrics
from src.financials.ratio_metrics import RatioMetrics
from src.financials.correlation_metrics import RollingCorrelation, SignCorrelation

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



### All data are artificial, even the public indexes

In [122]:
path_data = os.path.join("..", "data", "demo_data.csv")
demo_df = pd.read_csv(path_data)
demo_df.head()

,totalCashOutDelta,proceedsOfTheQuarter,fmv,returns,publicIndex1,publicIndex2,publicIndex3
0,54.494432,51.684306,9927.551507,0.003001,0.158101,0.064400,0.261923
1,-88.429400,391.414289,10229.424708,0.022513,0.153486,0.076818,0.309118
2,1027.837411,135.458283,11213.974988,0.037471,0.166384,0.076838,0.300997
3,565.458003,669.352861,10691.690203,0.018459,0.154188,0.083863,0.263816
4,461.772845,137.155926,13226.006855,0.014014,0.118355,0.074670,0.229321



### Preprocessing data 


### Computing financial rations with FinancialReport

FinancialReport shows the values for metrics irr, dvpi, rvpi, tvpi and moic for every quarter of the fund

In addition, FinancialReport has a plotting function integrated to plot line plots of metrics

In [123]:
financial_report_demo = FinancialReport(demo_df.totalCashOutDelta, demo_df.proceedsOfTheQuarter, demo_df.fmv)
financial_ratios = financial_report_demo.compute_all_ratios()
financial_ratios.head(20)

,dpi,rvpi,tvpi,moic,net_cashflows,irr
0,0.948433,182.175521,183.123954,1.432429,-2.810125,0.000000e+00
1,-13.057286,-301.441996,-314.499282,1.531944,477.033564,2.109291e+14
2,0.582106,11.282772,11.864879,1.692711,-415.345565,1.305374e+09
3,0.800270,6.856459,7.656729,1.713821,-311.450707,8.159398e+08
4,0.685292,6.543857,7.229148,2.097287,-636.067625,8.133604e+08
5,0.793680,5.989106,6.782786,1.910881,-404.940197,8.133415e+08
6,0.887751,6.154162,7.041913,1.926406,-213.927063,8.133414e+08
7,1.010473,5.519078,6.529551,1.845664,20.622787,8.133414e+08
8,0.890697,5.405995,6.296691,2.058321,-248.919098,8.133414e+08
9,0.971027,5.029481,6.000509,1.970229,-66.273466,8.133414e+08


In [124]:
#Plotting mechanism from FinancialReport
financial_report_demo.plot("net_cashflows", title="J Curve - Demo", name="Net Cash Position")


### Computing PME metrics with PmeTable and PmeMetrics

PmeMetrics is used to compute kspme and direct alpha. It is used internally by PmeTable to build a 3D array with dims (indexes, funds, metrics). 

PmeTable is used with a constructor from_data() that instantiates the class and producess the table in one action. 


In [125]:
pme = PmeMetrics(demo_df["totalCashOutDelta"], demo_df["proceedsOfTheQuarter"], demo_df["fmv"], demo_df["publicIndex1"])
pme.compute_ks_pme()

3.222948187435894

In [126]:
demo_df["entity"] = "Demo Data"
portfolio_list = demo_df["entity"].unique().tolist()
index_list = ["publicIndex1", "publicIndex2", "publicIndex3"]

#PmeTable uses PmeMetrics to compute several instances of PME metric for each fund with each index 
pme_table = PmeTable.from_long_data(demo_df, index_list, portfolio_list, "totalCashOutDelta", "proceedsOfTheQuarter", "fmv", discount_rate=0, metric="direct_alpha")
pme_table.table


### RatioMetrics for Sharpe and other ratios

Compute 5 metrics commonly used in finance to evaluate performances.

Uses a from_data() constructor to build the ratios. 

Requires returns at the minimum. For more complex ratios, require an index. Risk free rate is by default at 0.25% quarterly 

A function to convert absolute values in returns is included in the class and is activated when the flag is_return==False


In [127]:
#RatioMetrics has a constructor which is used for processing data before putting them in the class
ratio_metrics = RatioMetrics.from_data(demo_df["returns"], demo_df["publicIndex3"], risk_free_rate=0.0025, is_return=False)
ratio_metrics.compute_all_ratios()

{'sharpe_ratio': 0.9935150146410473,
 'sortino_ratio': 3.407769985433414,
 'modigliani_metric': 0.20711130741666844,
 'treynor_ratio': -0.2629871340185074,
 'jensen_alpha': 0.047294261695727875}


### Calculating rolling correlation with RollingCorrelation

Calculates the rolling correlation of an index against the market with a given window. 

Uses a from_data() constructor that produces a graph directly if show_plot=True. 

Requires an array of returns and a dataframe/array of public indexes to compare to. 

In [128]:
rollcorr = RollingCorrelation.from_data(demo_df["returns"], demo_df.loc[:, index_list], window_size=4, show_plot=False)
rollcorr.fig.update_layout(title="Pearson Correlation between demo fund and public indexes")


### Calculating Sign correlation with SignCorrelation

Calculates the Sign correlation of an index against the market. Inherits from RollingCorrelation and follows the same functionning.

Uses a from_data() constructor that produces a graph directly if show_plot=True. 

Requires an array of returns and a dataframe/array of public indexes to compare to. 

In [129]:
signcorr = SignCorrelation.from_data(demo_df["returns"], demo_df.loc[:, index_list], show_plot=False)
signcorr.fig.update_layout(title="Sign Correlation between demo fund and public indexes")